In [1]:
!pip install torch==2.0.1 torchtext==0.15.2


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from collections import Counter

# Определите модель
class SentimentModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_filters, filter_size, output_dim, dropout):
        super(SentimentModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv1d = nn.Conv1d(in_channels=embedding_dim, out_channels=num_filters, kernel_size=filter_size)
        self.fc = nn.Linear(num_filters, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.embedding(x)
        x = x.permute(0, 2, 1)  # (batch_size, seq_len, embedding_dim) -> (batch_size, embedding_dim, seq_len)
        x = self.conv1d(x)
        x = torch.relu(x)
        x = torch.max(x, dim=2)[0]  # Max pooling
        x = self.dropout(x)
        x = self.fc(x)
        return x

# Создайте обратный словарь из vocab
def get_index_to_token(vocab):
    try:
        index_to_token = vocab.get_itos()
    except AttributeError:
        index_to_token = {idx: token for token, idx in vocab.items()}
    return index_to_token

# Функция для создания словаря из данных
def build_vocab(data, max_size=10000):
    tokens = [token for text in data for token in text.split()]
    token_counts = Counter(tokens)
    most_common_tokens = token_counts.most_common(max_size - 2)
    vocab = {token: idx + 2 for idx, (token, _) in enumerate(most_common_tokens)}
    vocab['<unk>'] = 0  # Для неизвестных токенов
    vocab['<pad>'] = 1  # Для заполнения (padding)
    return vocab

# Создайте класс Dataset для обучения
class TrainTextDataset(Dataset):
    def __init__(self, csv_file, vocab, max_len):
        self.data = pd.read_csv(csv_file, header=0, names=['id', 'label', 'tweet'], dtype={'tweet': str})
        self.vocab = vocab
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = str(self.data.iloc[idx]['tweet'])
        label = self.data.iloc[idx]['label']
        tokens = text.split()[:self.max_len]  # Ограничиваем длину текста
        token_ids = [self.vocab[token] if token in self.vocab else self.vocab['<unk>'] for token in tokens]
        token_ids = token_ids + [self.vocab['<pad>']] * (self.max_len - len(token_ids))  # Padding
        return torch.tensor(token_ids), torch.tensor(label, dtype=torch.float32)

# Создайте класс Dataset для тестирования
class TestTextDataset(Dataset):
    def __init__(self, csv_file, vocab, max_len):
        self.data = pd.read_csv(csv_file, header=0, names=['id', 'tweet'], dtype={'tweet': str})
        self.vocab = vocab
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = str(self.data.iloc[idx]['tweet'])
        tokens = text.split()[:self.max_len]  # Ограничиваем длину текста
        token_ids = [self.vocab[token] if token in self.vocab else self.vocab['<unk>'] for token in tokens]
        token_ids = token_ids + [self.vocab['<pad>']] * (self.max_len - len(token_ids))  # Padding
        return torch.tensor(token_ids)

# Функция для вывода примеров и предсказаний
def show_predictions(model, data_loader, index_to_token, num_examples=5):
    model.eval()
    examples = []
    with torch.no_grad():
        for texts in data_loader:
            for i in range(min(num_examples, len(texts))):
                text = texts[i]
                prediction = model(text.unsqueeze(0)).squeeze(1)
                pred_label = torch.sigmoid(prediction).item()  # Применяем сигмоиду для получения вероятности
                pred_label = 1 if pred_label > 0.5 else 0  # Классификация по порогу
                tokens = [index_to_token.get(index.item(), '<unk>') for index in text if index != 0]
                text_str = ' '.join(tokens)
                examples.append((text_str, pred_label))
                if len(examples) >= num_examples:
                    break
            if len(examples) >= num_examples:
                break

    for i, (text, pred_label) in enumerate(examples):
        print(f"Example {i+1}:")
        print(f"Text: {text}")
        print(f"Predicted Label: {pred_label}")
        print("-" * 50)

# Гиперпараметры
embedding_dim = 50
num_filters = 100
filter_size = 3
output_dim = 1  # Бинарная классификация
dropout = 0.5
max_len = 100  # Максимальная длина последовательности

# Загрузка данных
train_data = pd.read_csv('/content/sample_data/train.csv', header=0, names=['id', 'label', 'tweet'], dtype={'tweet': str})
test_data = pd.read_csv('/content/sample_data/test.csv', header=0, names=['id', 'tweet'], dtype={'tweet': str})

# Создаем словарь на основе тренировочных данных
vocab = build_vocab(train_data['tweet'], max_size=10000)
index_to_token = get_index_to_token(vocab)
vocab_size = len(vocab)

# Инициализация модели, оптимизатора и критерия
model = SentimentModel(vocab_size, embedding_dim, num_filters, filter_size, output_dim, dropout)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.BCEWithLogitsLoss()

# Создание датасетов и загрузчиков данных
train_dataset = TrainTextDataset(csv_file='/content/sample_data/train.csv', vocab=vocab, max_len=max_len)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TestTextDataset(csv_file='/content/sample_data/test.csv', vocab=vocab, max_len=max_len)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Обучение модели
def train_model(model, data_loader, optimizer, criterion, num_epochs=5):
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for texts, labels in data_loader:
            optimizer.zero_grad()
            outputs = model(texts)
            loss = criterion(outputs.squeeze(1), labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(data_loader):.4f}')

train_model(model, train_loader, optimizer, criterion, num_epochs=5)

# Вывод примеров и предсказаний
show_predictions(model, test_loader, index_to_token)


Epoch 1/5, Loss: 0.4767
Epoch 2/5, Loss: 0.2864
Epoch 3/5, Loss: 0.2756
Epoch 4/5, Loss: 0.2709
Epoch 5/5, Loss: 0.2674
Example 1:
Text: #passion to find <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Predicted Label: 0
--------------------------------------------------
Example 2:
Text: @user #white want everyone to see the new #movie â and why <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <p

Что влияет на точность: 

Увеличение объема данных:

 Большее количество данных для тренировки обычно приводит к лучшей обобщающей способности модели.

Предварительная обработка текста:

Удаление стоп-слов, пунктуации, приведение к нижнему регистру и лемматизация/стемминг могли бы улучшить представление текста для модели.

Использование более качественного словаря (например, Word2Vec или GloVe embeddings) для представления слов вместо использования произвольного словаря с ограниченной длиной.

Оптимизация гиперпараметров:

Увеличение числа фильтров в свертке (Conv1D) может позволить модели лучше распознавать сложные паттерны в данных.
Использование более сложной архитектуры, такой как двуслойные или многослойные RNN или LSTM, может повысить способность модели запоминать контекст.

Регуляризация:

Dropout помогает бороться с переобучением, но его гиперпараметры (например, вероятность отключения нейронов) могут быть оптимизированы.
Аугментация данных:

Для текстовых данных можно использовать методы, такие как переводы, перефразирование или синтаксическая трансформация, чтобы увеличить объем данных.